In [1]:
from gymEnvs import *

from gymnasium.utils.play import PlayPlot, play

In [2]:
# Ising gym

env = IsingEnv(n=32,N=64,device="cpu")

key_to_action = dict(ms=1, s=2, w=3, mw=4,ma=5, a=6,d=7,md=8)

play(env,keys_to_action=key_to_action)

In [3]:
env.metadata.get("render_modes")

[]

In [3]:
o,_=env.reset()

In [31]:
o,r,d,*_=env.step(4)

In [32]:
np.max(o),np.min(o)

(255, 0)